In [1]:
import torch
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer

from sae import SaeConfig, SaeLayerRangeTrainer, TrainConfig
from sae.data import chunk_and_tokenize

%load_ext autoreload
%autoreload 2

In [2]:
MODEL = "EleutherAI/pythia-160m"
dataset = load_dataset(
    "togethercomputer/RedPajama-Data-1T-Sample",
    split="train",
    trust_remote_code=True,
).select(range(16))
tokenizer = AutoTokenizer.from_pretrained(MODEL)
tokenized = chunk_and_tokenize(dataset, tokenizer, max_seq_len=128)

gpt = AutoModelForCausalLM.from_pretrained(
    MODEL,
    device_map={"": "cuda"},
    torch_dtype=torch.bfloat16,
)

cfg = TrainConfig(
    SaeConfig(expansion_factor=16), batch_size=32
)
trainer = SaeLayerRangeTrainer(cfg, tokenized, gpt)

trainer.fit()

num_proc must be <= 16. Reducing num_proc to 16 for dataset of size 16.


Map (num_proc=16):   0%|          | 0/16 [00:00<?, ? examples/s]

Training on modules: [('gpt_neox.layers.0', 'gpt_neox.layers.1', 'gpt_neox.layers.2', 'gpt_neox.layers.3', 'gpt_neox.layers.4', 'gpt_neox.layers.5', 'gpt_neox.layers.6', 'gpt_neox.layers.7', 'gpt_neox.layers.8', 'gpt_neox.layers.9', 'gpt_neox.layers.10', 'gpt_neox.layers.11')]
Learning rate: 6.67e-05
bitsandbytes 8-bit Adam not available, using torch.optim.Adam
Run `pip install bitsandbytes` for less memory usage.
Number of SAE parameters: 2_718_065_664
Number of model parameters: 162_322_944


Training:   0%|          | 0/60 [00:00<?, ?it/s]

Saving checkpoint
